In [8]:
import time
#from numpy.lib.function_base import average
from qiskit import IBMQ, Aer, assemble, transpile, QuantumCircuit, ClassicalRegister, QuantumRegister
#from qiskit.visualization import array_to_latex
from qiskit_textbook.tools import array_to_latex
from qiskit.tools.monitor import job_monitor 
import numpy as np
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
import matplotlib.pyplot as plt
import pytz
from pytz import timezone
from datetime import datetime
from csv import writer
from scipy.stats import norm
from collections import defaultdict
import json

#Noise Model
def get_noise(p):                                                       #Not used with quantum backends, only with simulation
    error_meas = pauli_error([('X',p), ('I', 1 - p)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")      # measurement error is applied to measurements
       
    return noise_model

###---------Global Variables----------#
times = []
matrix_array_for_noise = []
normalized_matrix_array_for_noise = []
calibration_det_for_noise = []
datet = []



#IBMQ.save_account("d494816e01a737eb4ea87d4c9f6e2dcfdc7ed10d72ad68be30ecc7b5f66db445359005a848fc923f7ee96ba8909bffe76351ae2780cf53fd6a5f8d0ecc4fb491")
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')
backend_use = provider.get_backend('ibmq_manila')
#print("backend: ", backend)

num_shots = 2000

def time_of_interval(value,unit="s"):  #Function to create time interval depending on unit, set as seconds by default 
    if unit == "s":
        return value
    if unit == "m" or "min":
        return value * 60
    if unit == "h" or "hr" or "hour":
        return value * 60 * 60
 
    
t_i = time_of_interval(1)   #time in specififed unit of each interval where the function will execute

num_of_intervals = 25        #number of interval to be loop in t_i


                               #This function creates calibration matrix for a set of state vectors
calibration_matrix = []
for state in ['00','01','10','11']:                  #Four qubit states
            results = []
            qc = QuantumCircuit(2,2)
            if state[0]=='1':
                qc.x(1)
            if state[1]=='1':
                qc.x(0)
            qc.measure([0, 1],[0, 1])
            
            #copied from summer school code
            transpiled_qc = transpile(qc, backend=backend_use)
            assembled_qc = assemble(transpiled_qc)
            #job = backend_use.run(transpiled_qc, shots=num_shots)
            #job_monitor(job, interval=2)
            #results = job.result().get_counts()
            simulator = Aer.get_backend('aer_simulator')
            results = simulator.run(assembled_qc, noise_model=get_noise(0.01), shots=num_shots).result().get_counts()
            for state2 in ['00','01','10','11']:
                check = state2 in results.keys()
                if check == False:
                    results.update({state2 : 0})
                all_results = []
            list00 = []
            all_results.append(results)
            for n in all_results:                                               #Construct calibration matrix
                for m in ['00','01','10','11']:
                    list00.append(n.get(m)/num_shots) 
            calibration_matrix.append(list00)    


#SIMULATION
N = 2
epsilon = 0.5
mass = 1.0

#define gauge_kinetic to take in only epsilon
def gauge_kinetic(epsilon):
    #create a circuit, since this gate only ever works on 1 qubit, your circuit need only 1 qubit
    circuit=QuantumCircuit(1)
    #apply the correct gate to your circuit
    circuit.rx(-epsilon/2,0)
    #turn the circuit into a gate, U_kg
    U_kg = circuit.to_gate()
    #Name your gate something useful for looking at the circuits later
    U_kg.name = "U$_{Kg}$"
    #return your gate
    return U_kg

def fermion_mass(epsilon,mass,eta):
    circuit=QuantumCircuit(1)
    circuit.rz(-epsilon*mass * eta,0)
    U_m = circuit.to_gate()
    U_m.name = "U$_m$"
    return U_m

def fermion_hopping_opt2(epsilon,eta):
    circuit= QuantumCircuit(3)
    circuit.cx(0,2)
    circuit.h(0)
    circuit.cx(1,0)
    circuit.cx(0,2)
    circuit.rz(epsilon/4 * eta,0)
    circuit.rz(-epsilon/4 * eta,2)
    circuit.cx(0,2)
    circuit.cx(1,0)
    circuit.h(0)
    circuit.cx(0,2)
    U_fho2 = circuit.to_gate()
    U_fho2.name = "U$_{fho2}$"
    return U_fho2

#define your function to take in a dictionary counts
def get_mean_fermion_number(counts):
    #first define a variable mean and set to 0
    mean = 0
    #define a variable total_counts = sum(counts.values())
    total_counts = sum(counts.values())
    #perform a for loop over the elements in counts, using for example `s` as the loop variable
    for s in counts:
        #define a variable p which is the last element of the identifier in your loop variable
        #i.e. p = s[-1]
        p = s[-1]
        #check if p is a `1`.
        if p == '1':
            #if p == '1', then add to mean counts[s]/total_counts
            mean += 1./total_counts * counts[s]
    return mean

#define a function `get_bootstrap_error(counts)` which takes in a dictionary `counts`
def get_bootstrap_error(counts):
    #compute the nshots by using sum(counts.values())
    nshots=sum(counts.values())
    #For our statistics, we will resample the distribution 100 times, so set B=100
    B = 100
    #Extract the list of unique keys in your dictionary and store them as a variable k via
    # k = list(counts.keys())
    k = list(counts.keys())
    #For each key, we need to define a probability list prob via [counts[a]/nshots for a in k]
    prob = [counts[a]/nshots for a in k]
    #define an empty array means
    means = []
    #With all this setup, now perform a for loop over b in range(B)
    for b in range(B):
        #set a variable m=0
        m = 0
        #build a new set of samples using numpy.random.choice with the arguments k, size=nshots, and p=prob
        #this function will return nshots worth of new results with probability prob from the choices in k
        samples = np.random.choice(k, size=nshots, p=prob)
        #make a for loop over s in the list samples
        for s in samples:
            #set a new variable p = s[-1] to again extract the number of electrons
            p = s[-1]
            #check if p = `1`
            if p == '1':
                #add 1/nshots to m if true
                m += 1./nshots
        #append the results of m to means
        means.append(m)
    #return from the function the standard deviation of means via numpy.std(means)   
    return np.std(means)

counts=[]
sim_counts=[]
means=[]
errs=[]
sim_means=[]
sim_errs=[]
Ts=[]


#Make the for loop
for T in range(int(0/epsilon),int(3/epsilon)):
    #Append Ts and create your circuit
    Ts.append(T)
    qc = QuantumCircuit(2*N-1, 2*N-1)

    #state prep!
    qc.x(0)
    qc.h(0)
    


    #time evolution
    for t in range(T):
        for n in range(0,2*N,2):
            qc.append(fermion_mass(epsilon,mass,(-1)**(n/2+1)),[n])
        for l in range(1,2*N-1,2):
            qc.append(gauge_kinetic(epsilon),[l])
        for n in range(0,2*N-3,2):
            qc.append(fermion_hopping_opt2(epsilon, (-1)**(n/2)),[n,n+1,n+2])
      
    #add measurement steps
    qc.measure(range(3), range(3))

    
    #define your two backends
    sim_backend = Aer.get_backend('aer_simulator')
    #backend = provider.get_backend('ibm_lagos')
    
    #transpile your circuit for each backend
    sim_tc = transpile(qc, sim_backend, optimization_level=3)
    #tc = transpile(qc, backend, optimization_level=3)

    #run each of your jobs, name them differently
    sim_job = sim_backend.run(sim_tc, noise_model=get_noise(0.01), shots=num_shots) 
    #job = backend.run(tc)

    #append your results to sim_counts and counts.  Note that to access these later you will need counts[T] or sim_counts[T]
    sim_result = sim_job.result().get_counts()
    sim_counts.append(sim_result)
    #result= job.result()
    #counts.append(result.get_counts())
  
    #print(result.get_counts()) 

    #Compute the means and standard deviations and store them into the correct variables    
    sim_means.append(get_mean_fermion_number(sim_counts[T]))  
    #means.append(get_mean_fermion_number(counts[T]))  
    
    sim_errs.append(get_bootstrap_error(sim_counts[T]))
    #errs.append(get_bootstrap_error(counts[T]))
    
    #print out the time elapsed,means, and errors
    #print(T*epsilon, sim_means[T], sim_errs[T], means[T], errs[T])
    
    print(T*epsilon, sim_means[T], sim_errs[T])
    

inv_calibration = np.linalg.inv(calibration_matrix) 
array_to_latex(inv_calibration)
array_to_latex(np.dot(calibration_matrix,inv_calibration))

ibmqfactory.load_account:WARNING:2022-02-10 19:31:40,599: Credentials are already in use. The existing account in the session will be replaced.


0.0 0.47750000000000004 0.011380615097612201
0.5 0.468 0.010638961415476609
1.0 0.453 0.010367477996118448
1.5 0.464 0.011131154477411597
2.0 0.46699999999999997 0.010195463451947644
2.5 0.498 0.010262158642312454


<IPython.core.display.Math object>

<IPython.core.display.Math object>